# NYC CITY BIKE ANALYSIS FOR 2016-17

## Importing libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc 

pd.set_option("display.max_columns",30)
pd.set_option("display.max_rows",200)

## Load Dataset

In [2]:
server = 'LAPTOP-9D2D5CCL' 
database = 'data_science' 
conn_str = (
    r'Driver=SQL Server;'
    r'Server=LAPTOP-9D2D5CCL;'
    r'Database=data_science;'
    r'Trusted_Connection=yes;'
    )
cnxn = pyodbc.connect(conn_str)
cursor = cnxn.cursor()
# select data from SQL table to insert in dataframe.

# df = pd.read_sql(query, cnxn)
# print(df.head(10))

In [3]:
query_1 = "Select  * from [dbo].[citibike_tripdata_daily_fact] where year_name = 2016;"
df_chunks_1 = pd.read_sql_query(query_1, cnxn, chunksize=1000000)
def chunks_to_df(gen):
    chunks = []
    for df1 in gen:
        chunks.append(df1)
    return pd.concat(chunks).reset_index().drop('index', axis=1)

df1 = chunks_to_df(df_chunks_1)

In [4]:
print("Shape of data: ", df1.shape)

Shape of data:  (12680301, 19)


In [ ]:
query_2 = "Select  * from [dbo].[citibike_tripdata_daily_fact] where year_name = 2017;"
df_chunks_2 = pd.read_sql_query(query_2, cnxn, chunksize=1000000)
def chunks_to_df(gen):
    chunks = []
    for df2 in gen:
        chunks.append(df2)
    return pd.concat(chunks).reset_index().drop('index', axis=1)

df2 = chunks_to_df(df_chunks_2)

In [7]:
print("Shape of data: ", df1.shape)
print("Shape of data: ", df2.shape)

Shape of data:  (12680301, 19)
Shape of data:  (15076395, 19)


In [8]:
df = pd.concat([df1, df2])

# Set id as index
df = df.set_index("id")
data = df.copy

In [ ]:
print("Shape of data: ", df.shape)

In [ ]:
df.describe()

## Exploratory Data Analysis

In [ ]:
df.info()

In [ ]:
# Finding numeric data types
df.select_dtypes(include = ['int64','float64']).columns

In [ ]:
# Finding object data types
df.select_dtypes(include = ['object']).columns

In [ ]:
df.info()

In [ ]:
df.head(10)

### Handling Missing Values

In [ ]:
# Replace null age with meen
df['age'] = df['age'] .fillna((df['age'] .mean()))

In [ ]:
df['age'].isnull().sum()

In [ ]:
df.head(10)

In [ ]:
# Get the percentage of null values for each column
null_percent  = df.isnull().sum()/df.shape[0]*100
null_percent

In [ ]:
#  Replace Null values of usertype with mode
df['usertype'] = df['usertype'] .fillna((df['usertype'] .mode()[0]))
# Get the percentage of null values for each column
null_percent_usertype  = df['usertype'].isnull().sum()/df['usertype'].shape[0]*100
null_percent_usertype

In [ ]:
df.columns

### Checking Unique Values

In [ ]:
for i in df[['year_name', 'month_name', 'trip_start_date', 'gender', 'age',
       'is_weekend', 'usertype', 'start_station_id', 'start_station_name',
       'end_station_id', 'end_station_name']] :
    print(i + "\t" + str(len(df[i].unique())))

In [ ]:
# find unique values of each column
for i in df[['year_name', 'month_name', 'trip_start_date', 'gender', 'age',
       'is_weekend', 'usertype', 'start_station_id', 'start_station_name',
       'end_station_id', 'end_station_name']] :
    print ( "Unique value of : >>> {} ({}) \n{} \n".format(i,len(df[i].unique()),df[i].unique()))

## Outliers Treatment

### Outliers for Age

In [ ]:
# Plot the distplot of Age
plt.figure(figsize = (10,8))
bar = sns.distplot(df['age'])
bar.legend(["Skewness: {:.2f}".format(df['age'].skew())])

 As in evident from graph, most users are aged 40 while the age ranges from 16 to 80. Data points rangung from 80 to 160 are outliers and can be representing bad data. Age can't be 160 for a rider. Hence we will find out the data points above age 80 and treat the outliers.

In [ ]:
df[df['age']>80]['age'].value_counts().sort_values

In [ ]:
df[df['age']<=80]['age'].value_counts().sort_values

We can see that for age 80-85, trips are in the range of 1000-2000. Hence for all ages above 80, we will treat them as age 80

In [ ]:
df.loc[(df.age > 80),'age'] = 80

In [ ]:
df[df['age']>=79]['age'].value_counts().sort_values

## Analyze Monthly Data

In [ ]:
month_group = df.groupby(['year_name','month_name'])
year_group = df.groupby(['year_name'])
pd.set_option('display.float_format', lambda x: '%.5f' % x)

### Top 5 Destinations By Year

In [ ]:
year_group['end_station_name'].value_counts(normalize = True)

### Top 5 Starting Stations By Year

In [ ]:
year_group['start_station_name'].value_counts(normalize = True)

### Top 5 Destinations By Year and Month

In [ ]:
month_group['end_station_name'].value_counts(normalize = True)

###  Bikes Used in a day By Year

In [ ]:
year_group['cnt_bike'].agg(['max','min','mean'])

###  Bikes Used in a day By Month and Year

In [ ]:
month_group['cnt_bike'].agg(['max','min','mean'])

###  Trips Taken in a single day By Year

In [ ]:
year_group['cnt_trips'].agg(['max','min','mean'])

In [ ]:
df['year_name'].value_counts()

###  Trips Taken in a day By Month and Year

In [ ]:
month_group['cnt_trips'].agg(['max','min','mean'])

# KPIs By Year

In [ ]:
df.groupby(
   ['year_name']
).agg(
    {
         'cnt_trips':[min, max, sum, 'mean'],   
         'cnt_bike': [min, max, sum, 'mean'],  
         'avg_trip_dur_min': [min, max, sum, 'mean'] , 
         'avg_dist_km':  [min, max, sum, 'mean'] ,
         'speed_mps' :  [min, max, sum, 'mean'] ,
        'total_dist_km' :  [min, max, sum, 'mean'] ,
        'total_tripduration_min' : [min, max, sum, 'mean']
    }
)

### KPIs By Year and Month

In [ ]:
df.groupby(
   ['month_name','year_name']
).agg(
    {
         'cnt_trips':[min, max, sum, 'mean'],   
         'cnt_bike': [min, max, sum, 'mean'],  
         'avg_trip_dur_min': [min, max, sum, 'mean'] , 
         'avg_dist_km':  [min, max, sum, 'mean'] ,
         'speed_mps' :  [min, max, sum, 'mean'] ,
        'total_dist_km' :  [min, max, sum, 'mean'] ,
        'total_tripduration_min' : [min, max, sum, 'mean']
    }
)

### KPIs By Year : Weekend Analysis


In [ ]:
df.groupby(
   ['year_name','is_weekend']
).agg(
    {
         'cnt_trips':[min, max, sum, 'mean'],   
         'cnt_bike': [min, max, sum, 'mean'],  
         'avg_trip_dur_min': [min, max, sum, 'mean'] , 
         'avg_dist_km':  [min, max, sum, 'mean'] ,
         'speed_mps' :  [min, max, sum, 'mean'] ,
        'total_dist_km' :  [min, max, sum, 'mean'] ,
        'total_tripduration_min' : [min, max, sum, 'mean']
    }
)

# KPIs By Year and Month : Weekend Analysis


In [ ]:
df.groupby(
   ['month_name','year_name','is_weekend']
).agg(
    {
         'cnt_trips':[min, max, sum, 'mean'],   
         'cnt_bike': [min, max, sum, 'mean'],  
         'avg_trip_dur_min': [min, max, sum, 'mean'] , 
         'avg_dist_km':  [min, max, sum, 'mean'] ,
         'speed_mps' :  [min, max, sum, 'mean'] ,
        'total_dist_km' :  [min, max, sum, 'mean'] ,
        'total_tripduration_min' : [min, max, sum, 'mean']
    }
)

### KPIs By Year : Age Analysis


In [ ]:
df.groupby(
   ['year_name','age']
).agg(
    {
         'cnt_trips':[min, max, sum, 'mean'],   
         'cnt_bike': [min, max, sum, 'mean'],  
         'avg_trip_dur_min': [min, max, sum, 'mean'] , 
         'avg_dist_km':  [min, max, sum, 'mean'] ,
         'speed_mps' :  [min, max, sum, 'mean'] ,
        'total_dist_km' :  [min, max, sum, 'mean'] ,
        'total_tripduration_min' : [min, max, sum, 'mean']
    }
)

# KPIs By Year and Month : Age Analysis

In [ ]:
df.groupby(
   ['month_name','year_name','age']
).agg(
    {
         'cnt_trips':[min, max, sum, 'mean'],   
         'cnt_bike': [min, max, sum, 'mean'],  
         'avg_trip_dur_min': [min, max, sum, 'mean'] , 
         'avg_dist_km':  [min, max, sum, 'mean'] ,
         'speed_mps' :  [min, max, sum, 'mean'] ,
        'total_dist_km' :  [min, max, sum, 'mean'] ,
        'total_tripduration_min' : [min, max, sum, 'mean']
    }
)